In [1]:
import torch
import random

from sent_order.conll import Corpus, Document

from torch.nn import functional as F
from itertools import islice
from boltons.iterutils import pairwise, chunked
from tqdm import tqdm_notebook
from textblob import TextBlob

from IPython.core.display import display, HTML

In [2]:
model = torch.load(
    '../../data/coref.9.bin',
    map_location={'cpu': 'cpu'},
)

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'sent_order.models.coref.DocEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [3]:
model.eval()

Coref(
  (encode_doc): DocEncoder(
    (embeddings): WordEmbedding(39414, 300)
    (lstm): LSTM(300, 200, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (score_spans): SpanScorer(
    (attention): Scorer(
      (score): Sequential(
        (0): Linear(in_features=400, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
    (width_embeddings): DistanceEmbedding(9, 20)
    (sm): Scorer(
      (score): Sequential(
        (0): Linear(in_features=1120, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
  )
  (score_pairs): PairScorer(
    (dist_embeddings): DistanceEmbedding(9, 20)
    (sa): Scorer(
      (score): Sequential(
        (0): Linear(

In [4]:
c = Corpus.from_combined_file('../../data/development.conll')

In [18]:
colors = ('blue', 'green', 'red', 'orange', 'purple')

def viz_clusters(doc):
    
    clusters = model.predict(doc)
    spans = model(doc)
    
    tags = {
        i: tag
        for tag, c in enumerate(clusters)
        for i1, i2 in c
        for i in range(i1, i2+1)
    }
    
    tokens = []
    for i, token in enumerate(doc.tokens):
        tag = tags.get(i)
        color = colors[tag] if tag is not None else 'black'
        weight = 'bold' if tag is not None else 'regular'
        tokens.append(f'<span style="color: {color}; font-weight: {weight};">{token.text}</span>')
        
    display(HTML(' '.join(tokens)))
    
    print(sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans]).item())
    print(sum(s.sm for s in spans))

In [20]:
viz_clusters(Document.from_text('David woke up. Then he went to the store.'))

0.996924877166748
tensor(-3.6794)


In [21]:
viz_clusters(Document.from_text('Then he went to the store. David woke up'))

0.3340340852737427
tensor(-3.3308)
